In [1]:
import numpy as np
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm import tqdm


import os
import psycopg2

from astropy import units as u
from astropy.table import Table, join, unique, vstack
from astropy.coordinates import SkyCoord, match_coordinates_sky, search_around_sky

from desitarget.targets import decode_targetid, encode_targetid, resolve
from desitarget.io import releasedict, release_to_photsys

from tqdm.notebook import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


from astropy.table import Table, vstack, hstack

import psycopg2

from tqdm.notebook import tqdm_notebook

import time

In [2]:
t = Table.read('/pscratch/sd/n/nravi/pv_rot_curves/iron_not_jura.fits')
t[:5]

TARGETID,SURVEY,FILENAME
float64,bytes4,bytes65
1079550230396930.0,sv3,iron/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits
1092744369930242.0,sv3,iron/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits
1079550234591232.0,sv3,iron/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits
1092744374124544.0,sv3,iron/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits
1092755086376960.0,sv3,iron/healpix/sv3/dark/103/10379/redrock-sv3-dark-10379.fits


In [31]:
def get_tf_targets_modified(redux, targetid=None, use_cached=False, verbose=False, with_static_pv=False, in_hp_fm=True, in_hp_rd=True):
    """Get TF targets from the DESI observations DB for a given spectroscopic reduction.
    
    Parameters
    ----------
    redux : str
        Spectroscopic reduction. E.g., 'everest', 'fuji', ...
    use_cached : bool
        Use cached data rather than re-running the query.
    
    Returns
    -------
    
    tf_targets : Table
        Table of Tully-Fisher observations.
    """
    tf_targets = None
    
    rows=[]

    if os.path.exists(f'tf_targets_{redux}.fits') and use_cached:
        tf_targets = Table.read('cache/tf_targets_{redux}.fits')
    else:
        try:
            db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi', password='')
            cursor = db.cursor()

            
            if in_hp_fm and in_hp_rd and with_static_pv:
                query = f"""SELECT rdx.targetid, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, rdx.sv1_desi_target, rdx.sv1_bgs_target, rdx.sv1_mws_target, rdx.sv1_scnd_target,
                rdx.sv2_desi_target, rdx.sv2_bgs_target, rdx.sv2_mws_target, rdx.sv2_scnd_target, rdx.sv3_desi_target, rdx.sv3_bgs_target, rdx.sv3_mws_target, rdx.sv3_scnd_target,
                rdx.cmx_target, rdx.priority_init, rdx.desi_target, rdx.bgs_target, rdx.mws_target, rdx.scnd_target, pv.pvtype, pv.sga_id
                   FROM {redux}.healpix_fibermap as rdx, static.pv as pv, {redux}.healpix_redshifts as zd
                   WHERE (rdx.targetid = {targetid} AND zd.targetid = {targetid} AND q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.));"""
            
            elif in_hp_fm and in_hp_rd:
                query = f"""SELECT rdx.targetid, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, rdx.sv1_desi_target, rdx.sv1_bgs_target, rdx.sv1_mws_target, rdx.sv1_scnd_target,
                rdx.sv2_desi_target, rdx.sv2_bgs_target, rdx.sv2_mws_target, rdx.sv2_scnd_target, rdx.sv3_desi_target, rdx.sv3_bgs_target, rdx.sv3_mws_target, rdx.sv3_scnd_target,
                rdx.cmx_target, rdx.priority_init, rdx.desi_target, rdx.bgs_target, rdx.mws_target, rdx.scnd_target
                   FROM {redux}.healpix_fibermap as rdx, static.pv as pv, {redux}.healpix_redshifts as zd
                   WHERE (rdx.targetid = {targetid} AND zd.targetid = {targetid});"""
            
            elif in_hp_fm:
                query = f"""SELECT rdx.targetid, rdx.sv1_desi_target, rdx.sv1_bgs_target, rdx.sv1_mws_target, rdx.sv1_scnd_target,
                rdx.sv2_desi_target, rdx.sv2_bgs_target, rdx.sv2_mws_target, rdx.sv2_scnd_target, rdx.sv3_desi_target, rdx.sv3_bgs_target, rdx.sv3_mws_target, rdx.sv3_scnd_target,
                rdx.cmx_target, rdx.priority_init, rdx.desi_target, rdx.bgs_target, rdx.mws_target, rdx.scnd_target
                   FROM {redux}.healpix_fibermap as rdx
                   WHERE rdx.targetid = {targetid};"""
                
            
            
            # query = f"""SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, rdx.mean_fiber_ra, rdx.mean_fiber_dec, rdx.std_fiber_ra, rdx.std_fiber_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, pv.pvtype, pv.sga_id
            #        FROM {redux}.healpix_fibermap as rdx, static.pv as pv, {redux}.healpix_redshifts as zd
            #        WHERE q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.) 
            #              AND zd.targetid = rdx.targetid;"""
            #              # AND pv.sga_id IS NOT NULL AND (pv.pvtype LIKE 'TFT' or pv.pvtype LIKE 'EXT' or pv.pvtype LIKE 'SGA');"""
            
#             query = f"""SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, rdx.mean_fiber_ra, rdx.mean_fiber_dec, rdx.std_fiber_ra, rdx.std_fiber_dec, pv.pvtype, pv.sga_id
#                     FROM {redux}.healpix_fibermap as rdx, static.pv as pv
#                     WHERE (rdx.targetid = {targetid} AND q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.));"""
            
            # query = f"""SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, rdx.mean_fiber_ra, rdx.mean_fiber_dec, rdx.std_fiber_ra, rdx.std_fiber_dec
            #         FROM {redux}.healpix_fibermap as rdx
            #         WHERE rdx.targetid = {targetid};"""
            
            # query = f"""SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, rdx.mean_fiber_ra, rdx.mean_fiber_dec, rdx.std_fiber_ra, rdx.std_fiber_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, rdx.release, rdx.
            #        FROM {redux}.healpix_fibermap as rdx, static.pv as pv, {redux}.healpix_redshifts as zd
            #        WHERE (zd.targetid = {targetid} AND rdx.targetid = {targetid});"""
                         # AND pv.sga_id IS NOT NULL AND (pv.pvtype LIKE 'TFT' or pv.pvtype LIKE 'EXT' or pv.pvtype LIKE 'SGA');"""

       
            # query = f"""SELECT rdx.targetid, rdx.release, rdx.sv1_desi_target, rdx.sv1_bgs_target, rdx.sv1_mws_target, rdx.
            #        FROM {redux}.healpix_fibermap as rdx, static.pv as pv, {redux}.healpix_redshifts as zd
            #        WHERE (zd.targetid = {targetid} AND rdx.targetid = {targetid});"""
            #              # AND pv.sga_id IS NOT NULL AND (pv.pvtype LIKE 'TFT' or pv.pvtype LIKE 'EXT' or pv.pvtype LIKE 'SGA');"""
             
            if verbose:
                print(query)

            cursor.execute(query)
            rows = cursor.fetchall()
#             tf_targets = Table(list(map(list, zip(*rows))),
#                                names=['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'MEAN_FIBER_RA', 'MEAN_FIBER_DEC','STD_FIBER_RA', 'STD_FIBER_DEC'])

#             #- Select only targets with SGA IDs and PV types matching SGA, EXT, and TFT
#             print('selecting')
#             select = (tf_targets['SGA_ID'] != None) & (tf_targets['TARGETID'] == targetid) &\
#                      ((tf_targets['PVTYPE'] == 'TFT') | \
#                       (tf_targets['PVTYPE'] == 'EXT') | \
#                       (tf_targets['PVTYPE'] == 'SGA') )
#             tf_targets = tf_targets[select]
#             print('done')
#             #- Use TARGETID to extract the photometric system used during targeting
#             _, _, releases, _, _, _ = decode_targetid(tf_targets['TARGETID'].value)

#             photsys = []
#             for i, release in enumerate(releases):
#                 ps = None

#                 if release in releasedict:
#                     ps = release_to_photsys([release])[0].decode('utf-8')
#                 else:
#                     #- Fall-through case: not all SGA center observations are in the main survey.
#                     #  In this case, select 'N' or 'S' based on the SGA object's position.
#                     ra  = tf_targets['TARGET_RA'][i]
#                     dec = tf_targets['TARGET_DEC'][i]
#                     c = SkyCoord(ra=ra, dec=dec, unit='degree')

#                     #- N: in galactic northern hemisphere and with dec > 32.375. Else, S.
#                     isnorth = (c.galactic.b > 0) & (dec > 32.375)
#                     ps = 'N' if isnorth else 'S'

#                 photsys.append(ps)

#             #- Complain if the photsys table doesn't match the size of the Vrot table.
#             if len(photsys) != len(tf_targets):
#                 print(f'photsys array of len {len(photsys)} != targets array of len {len(tf_targets)}')

#             tf_targets['PHOTSYS'] = photsys

#             # tf_targets.write(f'cache/tf_targets_{redux}.fits', overwrite=True)

        except Exception as error:
            print(error)
        finally:
            if db is not None:
                db.close()

    return rows

In [32]:
jura = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/Y3/desi_pv_jura_healpix.fits')

In [34]:
in_jura = 0
not_in_jura = 0
matches=[]
for i in tqdm(range(len(t))):
    match = get_tf_targets_modified('jura', targetid=int(t['TARGETID'][i]), with_static_pv=True)
    if len(match) > 0:
        in_jura +=1
        matches.append(match)
    else:
        not_in_jura +=1

100%|██████████| 747/747 [00:37<00:00, 19.86it/s]


In [36]:
print('with pv')
print(in_jura)
print(not_in_jura)

with pv
0
747


In [ ]:
#check iron too
in_iron = 0
not_in_iron = 0
for i in tqdm(range(len(t))):
    match = get_tf_targets_modified('iron', targetid=int(t['TARGETID'][i]))
    if len(match) > 0:
        in_iron +=1
    else:
        not_in_iron +=1

  1%|▏         | 10/747 [01:08<1:27:10,  7.10s/it]

In [ ]:
print('iron healpix in iron db')
print(in_iron)
print(not_in_iron)

In [5]:
in_jura = 0
not_in_jura = 0
for i in tqdm(range(len(t))):
    match = get_tf_targets_modified('jura', targetid=int(t['TARGETID'][i]))
    if len(match) > 0:
        in_jura +=1
    else:
        not_in_jura +=1

100%|██████████| 747/747 [00:09<00:00, 81.01it/s] 


In [6]:
print('just healpix_fibermap')
print(in_jura)
print(not_in_jura)


just healpix_fibermap
23
724


In [16]:
in_jura = 0
not_in_jura = 0
for i in tqdm(range(len(t))):
    match = get_tf_targets_modified('jura', targetid=int(t['TARGETID'][i]))
    if len(match) > 0:
        in_jura +=1
    else:
        not_in_jura +=1

100%|██████████| 747/747 [03:11<00:00,  3.90it/s]


In [17]:
print('healpix_fibermap and healpix_redshifts')
print(in_jura)
print(not_in_jura)

healpix_fibermap and healpix_redshifts
23
724


In [ ]:
# what program are the 23 in
# remaining 724 don't know why theyre missing



In [18]:
iron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/desi_pv_tf_iron_healpix.fits')


In [20]:
bad_iron = iron[iron['TARGETID']<0]

In [26]:
bad_iron

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64
-205733115,193.87126208164,-7.79797192944254,25775,main,0.0130934193382867,7.45083636540879e-05,512,83.9020205204843,iron/healpix/main/bright/257/25775/redrock-main-bright-25775.fits,EXT,1232665,193.8711697701549,-7.797829020837202
-15733115,193.871320929678,-7.79798768021886,25775,main,0.013130791151608,8.25625309823356e-05,512,391.830742861355,iron/healpix/main/dark/257/25775/redrock-main-dark-25775.fits,EXT,1232665,193.8711697701549,-7.797829020837202
-256075243,267.712765090224,29.4851124724098,32659,main,0.0249376137955555,2.44308294845057e-05,2560,1544.51990368162,iron/healpix/main/bright/326/32659/redrock-main-bright-32659.fits,SGA,447725,267.7127798938763,29.484990855307803
-236269010,159.441551301486,11.8164401561333,27589,main,0.0536937948152313,2.00541500335771e-06,2560,17253.9541264772,iron/healpix/main/bright/275/27589/redrock-main-bright-27589.fits,SGA,112426,159.44167440686755,11.816368587372878
-215264164,139.654305794728,0.327220629540815,27301,main,0.169394003714603,1.66655025062609e-05,2560,1103.07280277461,iron/healpix/main/bright/273/27301/redrock-main-bright-27301.fits,SGA,460051,139.65430588469135,0.32746260142544686
-25264164,139.654293661988,0.327224885478571,27301,main,0.169385282648763,1.45650882820012e-05,2560,2406.85880166152,iron/healpix/main/dark/273/27301/redrock-main-dark-27301.fits,SGA,460051,139.65430588469135,0.32746260142544686
-204141114,256.286226415617,7.99070771912205,31568,main,0.030854589838939,1.01156963564606e-05,2560,18554.693482786,iron/healpix/main/bright/315/31568/redrock-main-bright-31568.fits,SGA,1127864,256.2861272736802,7.990563879458973
-14141114,256.286208378824,7.99067911276296,31568,main,0.0308607643018934,5.19829767772308e-06,512,65773.1641197163,iron/healpix/main/dark/315/31568/redrock-main-dark-31568.fits,SGA,1127864,256.2861272736802,7.990563879458973
-83744284,192.08201456717,-0.515340597093489,26216,main,0.932752477574551,2.23214609614882e-46,518,0.0,iron/healpix/main/dark/262/26216/redrock-main-dark-26216.fits,SGA,821777,192.08195042935628,-0.515288210200595


In [27]:
in_jura=0
not_in_jura =0
for i in tqdm(range(len(bad_iron))):
    match = get_tf_targets_modified('jura', targetid=int(bad_iron['TARGETID'][i]))
    if len(match) > 0:
        in_jura +=1
    else:
        not_in_jura +=1

100%|██████████| 30/30 [03:16<00:00,  6.54s/it]


In [28]:
print('bad iron in jura')
print(in_jura)
print(not_in_jura)

bad iron in jura
28
2
